In [1]:
# require(data.table)
# require(bit64)
# require(dbscan)
# require(doParallel)
# require(rBayesianOptimization)
# path='../input/train_1/'
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

from trackml.dataset import load_event, load_dataset
from trackml.score import score_event
from trackml.randomize import shuffle_hits

from sklearn.preprocessing import StandardScaler
import hdbscan as _hdbscan
from scipy import stats
from tqdm import tqdm

import time

from sklearn.cluster.dbscan_ import dbscan
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

from sklearn.neighbors import KDTree
import hdbscan
from bayes_opt import BayesianOptimization

# https://www.ellicium.com/python-multiprocessing-pool-process/
# http://sebastianraschka.com/Articles/2014_multiprocessing.html
from multiprocessing import Pool

In [2]:
import os
import time
import hdbscan as _hdbscan
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

In [3]:
def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits.hit_id.values, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission

In [4]:
def preprocess(hits):        
    x = hits.x.values
    y = hits.y.values
    z = hits.z.values

    r = np.sqrt(x**2 + y**2 + z**2)
    hits['x2'] = x/r
    hits['y2'] = y/r

    r = np.sqrt(x**2 + y**2)
    hits['z2'] = z/r

    ss = StandardScaler()
    X = ss.fit_transform(hits[['x2', 'y2', 'z2']].values)
#     for i, rz_scale in enumerate(self.rz_scales):
#         X[:,i] = X[:,i] * rz_scale

    return X

In [5]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

#------------------------------------------------------

def make_counts(labels):
    
    
    _,reverse,count = np.unique(labels,return_counts=True,return_inverse=True)
    counts = count[reverse]
    counts[labels==0]=0
    
    return counts

def one_loop(param):
    
    # <todo> tune your parameters or design your own features here! 
    
    i,m, x,y,z, d,r, a, a_start,a_step = param
    #print('\r %3d  %+0.8f '%(i,da), end='', flush=True)
    
    da = m*(a_start - (i*a_step))
    aa = a + np.sign(z)*z*da 
    zr = z/r
    
    X = StandardScaler().fit_transform(np.column_stack([aa, aa/zr, zr, 1/zr, aa/zr + 1/zr]))
    _,l = dbscan(X, eps=0.0035, min_samples=1,)

    return l

def one_loop1(param):
    
    # <todo> tune your parameters or design your own features here! 
    
    i,m, x,y,z, d,r,r2,z2,a, a_start,a_step = param
    #print('\r %3d  %+0.8f '%(i,da), end='', flush=True)
    
    da = m*(a_start - (i*a_step))
    aa = a + np.sign(z)*z*da
#     if m == 1:
#         print(da)
    zr = z/r # this is cot(theta), 1/zr is tan(theta)
    theta = np.arctan2(r, z)
    ct = np.cos(theta)
    st = np.sin(theta)
    tt = np.tan(theta)
#     ctt = np.cot(theta)
    z2r = z2/r
    z2r2 = z2/r2
#     X = StandardScaler().fit_transform(df[['r2', 'theta_1', 'dip_angle', 'z2', 'z2_1', 'z2_2']].values)

    caa = np.cos(aa)
    saa = np.sin(aa)
    taa = np.tan(aa)
    ctaa = 1/taa
    
#     0.000005
    deps = 0.0000025
    X = StandardScaler().fit_transform(np.column_stack([caa, saa, tt, 1/tt]))
    l= DBSCAN(eps=0.0035+i*deps,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
#     _,l = dbscan(X, eps=0.0035, min_samples=1,algorithm='auto')
    
    return l

def one_loop2(param):
    
    # <todo> tune your parameters or design your own features here! 
    
    i,m, x,y,z, d,r,r2,z2,a, a_start,a_step = param
    #print('\r %3d  %+0.8f '%(i,da), end='', flush=True)
    
    da = m*(a_start - (i*a_step))
    aa = a + np.sign(z)*z*da
#     if m == 1:
#         print(da)
    zr = z/r # this is cot(theta), 1/zr is tan(theta)
    theta = np.arctan2(r, z)
    ct = np.cos(theta)
    st = np.sin(theta)
    tt = np.tan(theta)
#     ctt = np.cot(theta)
    z2r = z2/r
    z2r2 = z2/r2
#     X = StandardScaler().fit_transform(df[['r2', 'theta_1', 'dip_angle', 'z2', 'z2_1', 'z2_2']].values)

    caa = np.cos(aa)
    saa = np.sin(aa)
    taa = np.tan(aa)
    ctaa = 1/taa
    
#     0.000005
    deps = 0.0000025
    X = StandardScaler().fit_transform(np.column_stack([caa, saa, tt, 1/tt]))
    l= DBSCAN(eps=0.0035+i*deps,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
#     _,l = dbscan(X, eps=0.0035, min_samples=1,algorithm='auto')
    
    return l

def do_dbscan_predict(df):
    
    x  = df.x.values
    y  = df.y.values
    z  = df.z.values
    r  = np.sqrt(x**2+y**2)
    d  = np.sqrt(x**2+y**2+z**2)
    a  = np.arctan2(y,x)

    x2 = df['x']/d
    y2 = df['y']/d
    z2 = df['z']/r

    r2 = np.sqrt(x2**2 + y2**2)
    phi = np.arctan2(y, x)
    phi_deg= np.degrees(np.arctan2(y, x))
    phi2 = np.arctan2(y2, x2)
    phi2_deg = np.degrees(np.arctan2(y2, x2))
    
    for angle in range(-180,180,1):
    
        df1 = df.loc[(df.phi_deg>(angle-1.0)) & (df.phi_deg<(angle+1.0))]
    
        x  = df1.x.values
        y  = df1.y.values
        z  = df1.z.values
        r  = np.sqrt(x**2+y**2)
        d  = np.sqrt(x**2+y**2+z**2)
        a  = np.arctan2(y,x)

        x2 = df1['x']/d
        y2 = df1['y']/d
        z2 = df1['z']/r
    
        r2 = np.sqrt(x2**2 + y2**2)
        theta= np.arctan2(r, z)
        theta1 = np.arctan2(r2, z2)

    
        tan_dip = phi/theta
        tan_dip1 = phi/z2
        z2_1 = 1/z2
        z2_2 = phi/z2 + 1/z2

        dip_angle = np.arctan2(z2, (np.sqrt(x2**2 +y2**2)) * np.arccos(x2/np.sqrt(x2**2 + y2**2)))
        dip_angle1 = np.arctan2(z, (np.sqrt(x**2 +y**2)) * np.arccos(x2/np.sqrt(x**2 + y**2)))
        scores = []

        a_start,a_step,a_num = 0.00100,0.0000095,150
    
        params  = [(i,m, x,y,z,d,r,r2,z2, a, a_start,a_step) for i in range(a_num) for m in [-1,1]]

        if 1: 
            pool = Pool(processes=1)
            ls   = pool.map( one_loop1, params )


        if 0:
            ls = [ one_loop(param) for param in  params ]


        ##------------------------------------------------

        num_hits=len(df)
        labels = np.zeros(num_hits,np.int32)
        counts = np.zeros(num_hits,np.int32)
        for l in ls:
            c = make_counts(l)
            idx = np.where((c-counts>0) & (c<20))[0]
            labels[idx] = l[idx] + labels.max()
            counts = make_counts(labels)
       

#     cl = hdbscan.HDBSCAN(min_samples=1,min_cluster_size=7,
#                          metric='braycurtis',cluster_selection_method='leaf',algorithm='best', 
#                          leaf_size=50)
    
#     X = preprocess(df)
#     l1 = pd.Series(labels)
#     labels = np.unique(l1)
   
# #   print(X.shape)
# #   print(len(labels_org))
# #   print(len(labels_org[labels_org ==0]))
# #   print(len(labels_org[labels_org ==-1]))
    
#     n_labels = 0
#     while n_labels < len(labels):
#         n_labels = len(labels)
#         max_len = np.max(l1)
#         s = list(l1[l1 == 0].keys())
#         X = X[s]
#         print(X.shape)
#         if X.shape[0] <= 1:
#             break
#         l = cl.fit_predict(X)+max_len
# #         print(len(l))
#         l1[l1 == 0] = l
#         labels = np.unique(l1)
    
    return labels

## reference----------------------------------------------
def do_dbscan0_predict(df):
    x = df.x.values
    y = df.y.values
    z = df.z.values
    r = np.sqrt(x**2+y**2)
    d = np.sqrt(x**2+y**2+z**2)

    X = StandardScaler().fit_transform(np.column_stack([
        x/d, y/d, z/r]))
    _,labels = dbscan(X,
                eps=0.0075,
                min_samples=1,
                algorithm='auto',
                n_jobs=-1)

    #labels = hdbscan(X, min_samples=1, min_cluster_size=5, cluster_selection_method='eom')

    return labels

## reference----------------------------------------------
def do_dbscan0_predict(df):
    x = df.x.values
    y = df.y.values
    z = df.z.values
    r = np.sqrt(x**2+y**2)
    d = np.sqrt(x**2+y**2+z**2)

    X = StandardScaler().fit_transform(np.column_stack([
        x/d, y/d, z/r]))
    _,labels = dbscan(X,
                eps=0.0075,
                min_samples=1,
                algorithm='auto',
                n_jobs=-1)

    #labels = hdbscan(X, min_samples=1, min_cluster_size=5, cluster_selection_method='eom')

    return labels

def extend(submission,hits):
    df = submission.merge(hits,  on=['hit_id'], how='left')
#     df = submission.append(hits)
#     print(df.head())
#     dz = -5.5
#     df.z = df.z + dz
    df = df.assign(d = np.sqrt( df.x**2 + df.y**2 + df.z**2 ))
    df = df.assign(r = np.sqrt( df.x**2 + df.y**2))
    df = df.assign(arctan2 = np.arctan2(df.z, df.r))

    for angle in range(-180,180,1):

        print ('\r %f'%angle, end='',flush=True)
        #df1 = df.loc[(df.arctan2>(angle-0.5)/180*np.pi) & (df.arctan2<(angle+0.5)/180*np.pi)]
        df1 = df.loc[(df.arctan2>(angle-1.0)/180*np.pi) & (df.arctan2<(angle+1.0)/180*np.pi)]

        min_num_neighbours = len(df1)
        if min_num_neighbours<4: continue

        hit_ids = df1.hit_id.values
        x,y,z = df1.as_matrix(columns=['x', 'y', 'z']).T
        r  = (x**2 + y**2)**0.5
        r  = r/1000
        a  = np.arctan2(y,x)
        tree = KDTree(np.column_stack([a,r]), metric='euclidean')

        track_ids = list(df1.track_id.unique())
        num_track_ids = len(track_ids)
        min_length=3

        for i in range(num_track_ids):
            p = track_ids[i]
            if p==0: continue

            idx = np.where(df1.track_id==p)[0]
            if len(idx)<min_length: continue

            if angle>0:
                idx = idx[np.argsort( z[idx])]
            else:
                idx = idx[np.argsort(-z[idx])]


            ## start and end points  ##
            idx0,idx1 = idx[0],idx[-1]
            a0 = a[idx0]
            a1 = a[idx1]
            r0 = r[idx0]
            r1 = r[idx1]

            da0 = a[idx[1]] - a[idx[0]]  #direction
            dr0 = r[idx[1]] - r[idx[0]]
            direction0 = np.arctan2(dr0,da0) 

            da1 = a[idx[-1]] - a[idx[-2]]
            dr1 = r[idx[-1]] - r[idx[-2]]
            direction1 = np.arctan2(dr1,da1) 


            ## extend start point
            ns = tree.query([[a0,r0]], k=min(20,min_num_neighbours), return_distance=False)
            ns = np.concatenate(ns)
            direction = np.arctan2(r0-r[ns],a0-a[ns])
            ns = ns[(r0-r[ns]>0.01) &(np.fabs(direction-direction0)<0.04)]

            for n in ns:
                df.loc[ df.hit_id==hit_ids[n],'track_id' ] = p 

            ## extend end point
            ns = tree.query([[a1,r1]], k=min(20,min_num_neighbours), return_distance=False)
            ns = np.concatenate(ns)

            direction = np.arctan2(r[ns]-r1,a[ns]-a1)
            ns = ns[(r[ns]-r1>0.01) &(np.fabs(direction-direction1)<0.04)] 

            for n in ns:
                df.loc[ df.hit_id==hit_ids[n],'track_id' ] = p
    #print ('\r')
#     df = df[['particle_id', 'weight', 'event_id', 'hit_id', 'track_id']]
    df = df[['event_id', 'hit_id', 'track_id']]
    return df

In [6]:
def trackML(df, w1, w2, w3, Niter):
    x  = df.x.values
    y  = df.y.values
    z  = df.z.values
    rt  = np.sqrt(x**2+y**2)
    r  = np.sqrt(x**2+y**2+z**2)
    a0  = np.arctan2(y,x)

    
    phi = np.arctan2(y, x)
    phi_deg= np.degrees(np.arctan2(y, x))
    
    
    z1 = z/rt
    z2 = z/r
    
    theta = np.arctan2(rt, z)
    tt = np.tan(theta)
    
#     z3 = np.log1p(abs(z/r))*np.sign(z)
    
    mm = 1
    ls = []
    print(Niter)
    Niter = Niter.astype(np.int32)
    print(Niter)
    print(w1, w2, w3, Niter)
    for ii in range(Niter):
        mm = mm * (-1)
#         print(np.pi)
        a1 = a0+mm*(rt+0.0000075*rt**2)/1000*(ii/2)/180*np.pi
#         print(a0, a1)
        saa = np.sin(a1)
        caa = np.cos(a1)
        
        deps = 0.0000025
        X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2]))
#         print(X.shape)
        
        cx = [w1,w1,w2,w3, w4]
        for jj in range(X.shape[1]): 
             X[:,jj] = X[:,jj]*cx[jj]
        l= DBSCAN(eps=0.0035,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
        ls.append(l)
    mm = 1
#     for ii in range(Niter):
#         mm = mm * (1)
# #         print(np.pi)
#         a1 = a0+mm*(rt+0.000006*rt**2)/1000*(ii/2)/180*np.pi
# #         print(a0, a1)
#         saa = np.sin(a1)
#         caa = np.cos(a1)
        
#         deps = 0.0000025
#         X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2]))
# #         print(X.shape)
        
#         cx = [w1,w1,w2,w3]
#         for jj in range(X.shape[1]): 
#              X[:,jj] = X[:,jj]*cx[jj]
#         deps = 0.0000025
#         l= DBSCAN(eps=0.0035,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
#         ls.append(l)
        
        
    num_hits=len(df)
    labels = np.zeros(num_hits,np.int32)
    counts = np.zeros(num_hits,np.int32)
    for l in ls:
        c = make_counts(l)
        idx = np.where((c-counts>0) & (c<20))[0]
        labels[idx] = l[idx] + labels.max()
        counts = make_counts(labels)
        
    return labels

In [7]:
def trackML2(df, w1, w2, w3, Niter):
    x  = df.x.values
    y  = df.y.values
    z  = df.z.values
    
    dz = 0
    z = z + dz
    
    rt  = np.sqrt(x**2+y**2)
    r  = np.sqrt(x**2+y**2+z**2)
    a0  = np.arctan2(y,x)

    
    phi = np.arctan2(y, x)
    phi_deg= np.degrees(np.arctan2(y, x))
    
    
    z1 = z/rt
    z2 = z/r
    
    theta = np.arctan2(rt, z)
    tt = np.tan(theta)
    
    mm = 1
    ls = []
    
#     print(Niter)
#     Niter = Niter.astype(np.int32)
#     print(Niter)
#     print(w1, w2, w3, Niter)
    
    for ii in range(Niter):
        mm = mm * (-1)
#         print(np.pi)
#         a1 = a0+mm*(rt+0.000045*rt**2)/1000*(ii/2)/180*np.pi
        a1 = a0+mm*(rt+0.0000075*rt**2)/1000*(ii/2)/180*np.pi
#         print(a0, a1)
        saa = np.sin(a1)
        caa = np.cos(a1)
        
        deps = 0.0000025
        X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2]))
#         print(X.shape)
        
        cx = [w1,w1,w2,w3]
        for jj in range(X.shape[1]): 
             X[:,jj] = X[:,jj]*cx[jj]
        deps = 0.0000025
        l= DBSCAN(eps=0.0035,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
        ls.append(l)
    
#     for ii in range(Niter):
#         mm = mm * (1)
# #         print(np.pi)
#         a1 = a0+mm*(rt+0.000005*rt**2)/1000*(ii/2)/180*np.pi
# #         print(a0, a1)
#         saa = np.sin(a1)
#         caa = np.cos(a1)
        
#         deps = 0.0000025
#         X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2]))
# #         print(X.shape)
        
#         cx = [w1,w1,w2,w3]
#         for jj in range(X.shape[1]): 
#              X[:,jj] = X[:,jj]*cx[jj]
#         deps = 0.0000025
#         l= DBSCAN(eps=0.0035+ii*deps,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
#         ls.append(l)
        
    num_hits=len(df)
    labels = np.zeros(num_hits,np.int32)
    counts = np.zeros(num_hits,np.int32)
    for l in ls:
        c = make_counts(l)
        idx = np.where((c-counts>0) & (c<20))[0]
        labels[idx] = l[idx] + labels.max()
        counts = make_counts(labels)
        
    return labels

In [8]:
def trackML3(df, w1, w2, w3, w4, Niter):
    x  = df.x.values
    y  = df.y.values
    z  = df.z.values
    
    dz = 0
    z = z + dz
    
    rt  = np.sqrt(x**2+y**2)
    r  = np.sqrt(x**2+y**2+z**2)
    a0  = np.arctan2(y,x)

    
    phi = np.arctan2(y, x)
    phi_deg= np.degrees(np.arctan2(y, x))
    
    
    z1 = z/rt
    z2 = z/r
    
    theta = np.arctan2(rt, z)
    tt = np.tan(theta)
    
    mm = 1
    ls = []
    
#     print(Niter)
#     Niter = Niter.astype(np.int32)
#     print(Niter)
#     print(w1, w2, w3, Niter)
    
    for ii in range(Niter):
        mm = mm * (-1)
#         print(np.pi)
#         a1 = a0+mm*(rt+0.000045*rt**2)/1000*(ii/2)/180*np.pi
        a1 = a0+mm*(rt+0.0000075*rt**2)/1000*(ii/2)/180*np.pi
#         print(a0, a1)
        saa = np.sin(a1)
        caa = np.cos(a1)
        
        deps = 0.0000025
        X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2, rt/r]))
#         print(X.shape)
        
        cx = [w1,w1,w2,w3, w4]
        for jj in range(X.shape[1]): 
             X[:,jj] = X[:,jj]*cx[jj]
        deps = 0.0000025
        l= DBSCAN(eps=0.0035,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
        ls.append(l)
    
#     for ii in range(Niter):
#         mm = mm * (1)
# #         print(np.pi)
#         a1 = a0+mm*(rt+0.000005*rt**2)/1000*(ii/2)/180*np.pi
# #         print(a0, a1)
#         saa = np.sin(a1)
#         caa = np.cos(a1)
        
#         deps = 0.0000025
#         X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2]))
# #         print(X.shape)
        
#         cx = [w1,w1,w2,w3]
#         for jj in range(X.shape[1]): 
#              X[:,jj] = X[:,jj]*cx[jj]
#         deps = 0.0000025
#         l= DBSCAN(eps=0.0035+ii*deps,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
#         ls.append(l)
        
    num_hits=len(df)
    labels = np.zeros(num_hits,np.int32)
    counts = np.zeros(num_hits,np.int32)
    for l in ls:
        c = make_counts(l)
        idx = np.where((c-counts>0) & (c<20))[0]
        labels[idx] = l[idx] + labels.max()
        counts = make_counts(labels)
        
    return labels

In [9]:
def trackML4(df, w1, w2, w3, w4, Niter):
    x  = df.x.values
    y  = df.y.values
    z  = df.z.values
    
    dz = 0
    z = z + dz
    
    rt  = np.sqrt(x**2+y**2)
    r  = np.sqrt(x**2+y**2+z**2)
    a0  = np.arctan2(y,x)

    
    phi = np.arctan2(y, x)
    phi_deg= np.degrees(np.arctan2(y, x))
    
    
    z1 = z/rt
    z2 = z/r
    
    theta = np.arctan2(rt, z)
    tt = np.tan(theta)
    
    mm = 1
    ls = []
    
#     print(Niter)
#     Niter = Niter.astype(np.int32)
#     print(Niter)
#     print(w1, w2, w3, Niter)
    
    for ii in range(Niter):
        mm = mm * (-1)
#         print(np.pi)
#         a1 = a0+mm*(rt+0.000045*rt**2)/1000*(ii/2)/180*np.pi
        a1 = a0+mm*(rt+0.0000075*rt**2)/1000*(ii/2)/180*np.pi
#         print(a0, a1)
        saa = np.sin(a1)
        caa = np.cos(a1)
        
        deps = 0.0000025
        X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2, rt/r]))
#         print(X.shape)
        
        cx = [w1,w1,w2,w3, w4]
        for jj in range(X.shape[1]): 
             X[:,jj] = X[:,jj]*cx[jj]
        deps = 0.0000025
        l= DBSCAN(eps=0.0035,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
        ls.append(l)
    
#     for ii in range(Niter):
#         mm = mm * (1)
# #         print(np.pi)
#         a1 = a0+mm*(rt+0.000005*rt**2)/1000*(ii/2)/180*np.pi
# #         print(a0, a1)
#         saa = np.sin(a1)
#         caa = np.cos(a1)
        
#         deps = 0.0000025
#         X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2]))
# #         print(X.shape)
        
#         cx = [w1,w1,w2,w3]
#         for jj in range(X.shape[1]): 
#              X[:,jj] = X[:,jj]*cx[jj]
#         deps = 0.0000025
#         l= DBSCAN(eps=0.0035+ii*deps,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
#         ls.append(l)
        
    num_hits=len(df)
    labels = np.zeros(num_hits,np.int32)
    counts = np.zeros(num_hits,np.int32)
    for l in ls:
        c = make_counts(l)
        idx = np.where((c-counts>0) & (c<20))[0]
        labels[idx] = l[idx] + labels.max()
        counts = make_counts(labels)
        
    return labels

In [10]:
def trackML6(df, w1, w2, w3, w5, w6, w7, Niter):
    x  = df.x.values
    y  = df.y.values
    z  = df.z.values
    
    rt  = np.sqrt(x**2+y**2)
    
#     zr = (z+dj)/r  *0.1
#     zr = (z+w6)/rt  *0.1
#     dz = 0.015
    dz = w7
    zr = z + dz
    
    r  = np.sqrt(x**2+y**2+z**2)
    a0  = np.arctan2(y,x)

    
    phi = np.arctan2(y, x)
    phi_deg= np.degrees(np.arctan2(y, x))
    
    
    z1 = z/rt
    z2 = z/r
    
    z3 = np.log1p(abs(z/r))*np.sign(z)
    
    theta = np.arctan2(rt, z)
    tt = np.tan(theta)
    
    w1 = 1.1932215111905984
    w2 = 0.39740553885387364
    w3 = 0.3512647720585538
    w4 = 0.1470
    w5 = 0.1439
    Niter=179
    
    mm = 1
    ls = []
    
    for ii in range(Niter):
        mm = mm * (-1)
#         print(np.pi)
        a1 = a0+mm*(rt+0.0000075*rt**2)/1000*(ii/2)/180*np.pi
#         print(a0, a1)
        saa = np.sin(a1)
        caa = np.cos(a1)
        
        deps = 0.0000025
        X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, rt/r, z3, zr]))
#         print(X.shape)
        w4 = 0.1470
#         w6 = 1
        cx = [w1,w1,w2,w4, w5, w6]
        for jj in range(X.shape[1]): 
             X[:,jj] = X[:,jj]*cx[jj]
        l= DBSCAN(eps=0.0035,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_

        ls.append(l)
        
    num_hits=len(df)
    labels = np.zeros(num_hits,np.int32)
    counts = np.zeros(num_hits,np.int32)
    for l in ls:
        c = make_counts(l)
        idx = np.where((c-counts>0) & (c<20))[0]
        labels[idx] = l[idx] + labels.max()
        counts = make_counts(labels)
        
    return labels

In [11]:
def trackML31_old(df, w1, w2, w3, w4, w5, w6, w7, w8, Niter):
    x  = df.x.values
    y  = df.y.values
    z  = df.z.values
    
    dz = 0
    z = z + dz
    
    rt  = np.sqrt(x**2+y**2)
    r  = np.sqrt(x**2+y**2+z**2)
    a0  = np.arctan2(y,x)

    x2 = x/r
    y2 = y/r
    
    
    phi = np.arctan2(y, x)
    phi_deg= np.degrees(np.arctan2(y, x))
    
    
    z1 = z/rt
    z2 = z/r
    
    z3 = np.log1p(abs(z/r))*np.sign(z)
    
    x1 = x/rt
    y1 = y/rt
    
    theta = np.arctan2(rt, z)
    tt = np.tan(theta)
    
    mm = 1
    ls = []
    
#     print(Niter)
#     Niter = Niter.astype(np.int32)
#     print(Niter)
#     print(w1, w2, w3, Niter)
    
    for ii in range(Niter):
        mm = mm * (-1)
#         print(np.pi)
#         a1 = a0+mm*(rt+0.000045*rt**2)/1000*(ii/2)/180*np.pi
#         a1 = a0+mm*(rt+0.0000075*rt**2)/1000*(ii/2)/180*np.pi
        a1 = a0+mm*(rt+ 0.0000145*rt**2)/1000*(ii/2)/180*np.pi
#         if ii == 0:
#             print(rt)
#             print(rt ** 2)
#             print(rt ** 3)
#             print(0.0000145*rt**2)
#             print(w6*rt**3)
#             print(rt+ 0.0000145*rt**2 + w6*rt**3)
#         print(a0, a1)
        saa = np.sin(a1)
        caa = np.cos(a1)
        
        deps = 0.0000025
        X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2, rt/r, x/r, y/r, z3, y1]))
#         print(X.shape)
        
        cx = [w1,w1,w2,w3, w4, w5, w6, w7, w8]
        for jj in range(X.shape[1]): 
             X[:,jj] = X[:,jj]*cx[jj]
        deps = 0.0000025
        l= DBSCAN(eps=0.0035,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
        ls.append(l)
    
#     for ii in range(Niter):
#         mm = mm * (1)
# #         print(np.pi)
#         a1 = a0+mm*(rt+0.000005*rt**2)/1000*(ii/2)/180*np.pi
# #         print(a0, a1)
#         saa = np.sin(a1)
#         caa = np.cos(a1)
        
#         deps = 0.0000025
#         X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2]))
# #         print(X.shape)
        
#         cx = [w1,w1,w2,w3]
#         for jj in range(X.shape[1]): 
#              X[:,jj] = X[:,jj]*cx[jj]
#         deps = 0.0000025
#         l= DBSCAN(eps=0.0035+ii*deps,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
#         ls.append(l)
        
    num_hits=len(df)
    labels = np.zeros(num_hits,np.int32)
    counts = np.zeros(num_hits,np.int32)
    for l in ls:
        c = make_counts(l)
        idx = np.where((c-counts>0) & (c<20))[0]
        labels[idx] = l[idx] + labels.max()
        counts = make_counts(labels)
        
    return labels

In [12]:
import hdbscan
def trackML31(df, w1, w2, w3, w4, w5, w6, w7, w8, w9, w10, w11, Niter):
    x  = df.x.values
    y  = df.y.values
    z  = df.z.values
    
#     dz = -5.5
#     z = z + dz
    z = z + w11
    
    rt  = np.sqrt(x**2+y**2)
    r  = np.sqrt(x**2+y**2+z**2)
    a0  = np.arctan2(y,x)

    x2 = x/r
    y2 = y/r
    
    
    phi = np.arctan2(y, x)
    phi_deg= np.degrees(np.arctan2(y, x))
    
    
    z1 = z/rt
    z2 = z/r
    
    z3 = np.log1p(abs(z/r))*np.sign(z)
    
    x1 = x/rt
    y1 = y/rt
    
    y3 = np.log1p(abs(y/r))*np.sign(y)
    
    theta = np.arctan2(rt, z)
    theta_deg = np.degrees(np.arctan2(rt, z))
    tt = np.tan(theta_deg)
    
    z4 = np.sqrt(abs(z/rt))
    
    mm = 1
    ls = []
    
#     print(Niter)
#     Niter = Niter.astype(np.int32)
#     print(Niter)
#     print(w1, w2, w3, Niter)
    
    for ii in range(Niter):
        mm = mm * (-1)
#         print(np.pi)
#         a1 = a0+mm*(rt+0.000045*rt**2)/1000*(ii/2)/180*np.pi
#         a1 = a0+mm*(rt+0.0000075*rt**2)/1000*(ii/2)/180*np.pi
#         a1 = a0+mm*(rt+ 0.0000145*rt**2)/1000*(ii/2)/180*np.pi
        a1 = a0+mm*(rt+ 0.0000005*rt**2)/1000*(ii/2)/180*np.pi
#         print(a0, a1)
        saa = np.sin(a1)
        caa = np.cos(a1)
        
        t1 = theta+mm*(rt+ 0.8435*rt**2)/1000*(ii/2)/180*np.pi
        ctt = np.cos(t1)
        stt = np.sin(t1)
        ttt = np.tan(t1)
        deps = 0.0000025
#         X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2, rt/r, x/r, z3, ctt, stt]))
#         X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2, rt/r, x/r, y/r, z3, y1, y3, 
#                                                             ctt, stt]))
#         print(X.shape)
        X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2]))
        
#         cx = [w1,w1,w2,w3, w4, w5, w6, w7, w7]
#         cx = [w1,w1,w2,w3, w4, w5, w6, w7, w8, w9, w10, w10]
        cx = [w1,w1,w2,w3]
        for jj in range(X.shape[1]): 
             X[:,jj] = X[:,jj]*cx[jj]
        deps = 0.0000025
#         cl = hdbscan.HDBSCAN(min_samples=1,min_cluster_size=7,cluster_selection_method='leaf',metric='euclidean', leaf_size=50)
#         l = cl.fit_predict(X)+1
        
        l= DBSCAN(eps=0.004,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
#         l= DBSCAN(eps=w7,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
        ls.append(l)
    
        
    num_hits=len(df)
    labels = np.zeros(num_hits,np.int32)
    counts = np.zeros(num_hits,np.int32)
    for l in ls:
        c = make_counts(l)
        idx = np.where((c-counts>0) & (c<20))[0]
        labels[idx] = l[idx] + labels.max()
        counts = make_counts(labels)
        
    return labels

In [13]:
######################## 
### working function ###
# ########################
# trackML <- function(dfh,w1,w2,w3,Niter){
#   dfh[,s1:=hit_id]
#   dfh[,N1:=1L] 
#   dfh[,r:=sqrt(x*x+y*y+z*z)]
#   dfh[,rt:=sqrt(x*x+y*y)]
#   dfh[,a0:=atan2(y,x)]
#   dfh[,z1:=z/rt]
#   dfh[,z2:=z/r]
#   mm     <-  1
#   for (ii in 0:Niter) {
#     mm <- mm*(-1)
#     dfh[,a1:=a0+mm*(rt+0.000005*rt^2)/1000*(ii/2)/180*pi]
#     dfh[,sina1:=sin(a1)]
#     dfh[,cosa1:=cos(a1)]
#     dfs=scale(dfh[,.(sina1,cosa1,z1,z2)])
# 	cx <- c(w1,w1,w2,w3)
#     for (jj in 1:ncol(dfs)) dfs[,jj] <- dfs[,jj]*cx[jj]
#     res=dbscan(dfs,eps=0.0035,minPts = 1)
#     dfh[,s2:=res$cluster]
#     dfh[,N2:=.N, by=s2]
#     maxs1 <- max(dfh$s1)
#     dfh[,s1:=ifelse(N2>N1 & N2<20,s2+maxs1,s1)]
#     dfh[,s1:=as.integer(as.factor(s1))]
#     dfh[,N1:=.N, by=s1]    
#   }
#   return(dfh$s1)
# }

In [14]:
def Fun4BO1(w1,w2,w3,Niter):
    track_id = trackML(hits, w1,w2,w3,Niter)
    
    sum_score=0
    sum = 0
    submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
        data=np.column_stack(([int(event_id),]*len(hits), hits.hit_id.values, track_id))
    ).astype(int)
    for i in range(8):
        submission = extend(submission,hits)
        score = score_event(truth, submission)
        print('[%2d] score : %0.8f'%(i, score))
        sum_score += score
        sum += 1

    print('--------------------------------------')
    sc = sum_score/sum
    print(sc)
    
    return sc

In [15]:
def Fun4BO2(w4):
    
    w1 = 1.1932215111905984
    w2 = 0.39740553885387364
    w3 = 0.3512647720585538
    Niter=179
    track_id = trackML3(hits, w1,w2,w3,w4,Niter)
    
    sum_score=0
    sum = 0
    submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
        data=np.column_stack(([int(event_id),]*len(hits), hits.hit_id.values, track_id))
    ).astype(int)
    for i in range(8):
        submission = extend(submission,hits)
        score = score_event(truth, submission)
        print('[%2d] score : %0.8f'%(i, score))
        sum_score += score
        sum += 1

    print('--------------------------------------')
    sc = sum_score/sum
    print(sc)
    
    return sc

In [16]:
def Fun4BO6(w7):
    
    w1 = 1.1932215111905984
    w2 = 0.39740553885387364
    w3 = 0.3512647720585538
    w4 = 0.1470
    w5 = 0.1439
    w6 = 0.0028
    Niter=179
    Niter = 247
    track_id = trackML6(hits, w1,w2,w3,w5,w6,w7,Niter)
    
    sum_score=0
    sum = 0
    submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
        data=np.column_stack(([int(event_id),]*len(hits), hits.hit_id.values, track_id))
    ).astype(int)
    for i in range(8):
        submission = extend(submission,hits)
        score = score_event(truth, submission)
        print('[%2d] score : %0.8f'%(i, score))
        sum_score += score
        sum += 1

    print('--------------------------------------')
    sc = sum_score/sum
    print(sc)
    
    return sc

In [17]:
def Fun4BO(w1,w2,w3,Niter):
    track_id = trackML2(hits, w1,w2,w3,Niter)
    
    sum_score=0
    sum = 0
    submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
        data=np.column_stack(([int(event_id),]*len(hits), hits.hit_id.values, track_id))
    ).astype(int)
    for i in range(8):
        submission = extend(submission,hits)
        score = score_event(truth, submission)
        print('[%2d] score : %0.8f'%(i, score))
        sum_score += score
        sum += 1

    print('--------------------------------------')
    sc = sum_score/sum
    print(sc)
    
    return sc

In [18]:
#######################################
# function for Bayessian Optimization #
#   (needs lists: Score and Pred)     #
#######################################
# Fun4BO <- function(w1,w2,w3,Niter) { 
#    dfh$s1 <- trackML(dfh,w1,w2,w3,Niter)
#    sub=data.table(event_id=nev,hit_id=dfh$hit_id,track_id=dfh$s1)
#    sc <- score(sub,dft)
#    list(Score=sc,Pred=0)
# }

In [19]:
def Fun4BO21(w1,w2,w3, w11):
    
#     w1 = 1.1932215111905984
#     w2 = 0.39740553885387364
#     w3 = 0.3512647720585538
#     w4 = 0.1470
#     w5 = 0.01201
#     Niter=179

#     Niter=247
    
##    org w1 = 1.1932215111905984
#     w1 = 1.33
##    org w2 = 0.39740553885387364
#     w2 = 1.15
##    org w3 = 0.3512647720585538
    hits.z = hits.z - w11
    
    w4 = 0.1470
    w5 = 0.01201
    w6 = 0.0003864
    w7 = 0.0205
    w8 = 0.0049
    w9 = 0.00121
    w10 = 1.4930496676654575e-05
#     w11 = 1.3073
    Niter = 247
    print(w1,w11,w2,w3, Niter)
#     Niter = Niter.astype(np.int32)
    track_id = trackML31(hits, w1,w2,w3,w4,w5,w6, w7, w8, w9, w10, w11, Niter)
    
    sum_score=0
    sum = 0
    submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
        data=np.column_stack(([int(event_id),]*len(hits), hits.hit_id.values, track_id))
    ).astype(int)
    for i in range(8):
        submission = extend(submission,hits)
        score = score_event(truth, submission)
        print('[%2d] score : %0.8f'%(i, score))
        sum_score += score
        sum += 1

    print('--------------------------------------')
    sc = sum_score/sum
    print(sc)
    
    return sc

In [20]:
# def run_dbscan():

data_dir = '../data/train'

#     event_ids = [
#             '000001030',##
#             '000001025','000001026','000001027','000001028','000001029',
#     ]

event_ids = [
        '000001030',##

]

sum=0
sum_score=0
for i,event_id in enumerate(event_ids):
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    hits  = pd.read_csv(data_dir + '/event%s-hits.csv'%event_id)
    cells = pd.read_csv(data_dir + '/event%s-cells.csv'%event_id)
    truth = pd.read_csv(data_dir + '/event%s-truth.csv'%event_id)
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    
    truth = pd.merge(truth, particles, how='left', on='particle_id')
    hits = pd.merge(hits, truth, how='left', on='hit_id')
    
    
    
    w1 = [0.09, 1.2]
    w2 = [0.03, 0.7]
    w3 = [0.01, 0.4]
    w11 = [-5.5, 5.5]
    Niter = [200, 500]
    
#     w1 = 1.1932215111905984
#     w2 = 0.39740553885387364
#     w3 = 0.3512647720585538
#     w4 = 0.1470
#     w5 = 0.01201
#     w6 = 0.0003864
#     w7 = 0.0205
#     w8 = 0.0049
#     w9 = 0.00121
#     w10 = 1.4930496676654575e-05
    
#     bo = BayesianOptimization(Fun4BO,pbounds = {'w1':w1,'w2':w2,'w3':w3,'Niter':Niter})
#     bo.maximize(init_points = 3, n_iter = 20, acq = "ucb", kappa = 2.576)
#     w1 = 1.1932215111905984
#     w2 = 0.39740553885387364
#     w3 = 0.3512647720585538
    w4 = [0.1, 0.2] # 0.1470 -> 0.55690
    w4 = 0.1470
    w5 = [0.001, 1.2] # 0.7781 -> 0.55646, 0.7235 + N = 247 => 0.56025
    
#     Niter = 179
#     Niter = 247
    w5 = 0.01
#     for w6 in [0.012, 0.01201, 0.01202, 0.01203, 0.01204, 0.01205, 0.01206, 0.01207, 0.01208, 0.01209, 0.0121]:
#     EPS = 1e-12
#     for w3 in np.arange(0.01, 0.1, 0.005):
#         print(w3)
#         Fun4BO21(w3)
#     Fun4BO21(1)
#     Niter = [240, 480]
#     w8 = [0.0040, 0.007]
    bo = BayesianOptimization(Fun4BO21,pbounds = {'w1':w1, 'w2':w2, 'w3':w3, 'w11':w11})
    bo.maximize(init_points = 50, n_iter = 5, acq = "ucb", kappa = 2.576)

# x/y: 7 | 06m30s |    0.55302 |    0.0100 | 
# x/y: 0.001: 0.55949
# x/y: 0.0001: 0.55949
# x/y: 0.002: 0.55959
# x/y: 0.003: 0.55915
# x/y: 0.0025: 0.55925
# x/y: 0.0015: 0.55953
# x/r: 0.0015: 0.56186
# x/r: 0.002: 0.56334
# x/r: 0.0025: 0.563989
# x/r: 0.003: 0.56447
# x/r: 0.01: 0.569822
# x/r: 0.015: 0.56940
# x/r: 0.012: 0.5719
# x/r: 0.01201: 0.57192
# rt/r, x/r, y/r, z3: 0.57216
# x1: 11 | 07m30s |    0.57217 |    0.0001 | 
# y1: 15 | 07m23s |    0.57333 |    0.0050 | 
# y1: 23 | 07m37s |    0.57326 |    0.0049 | (0.004875221386488249)
# y1: 0.0049000000000000024: 0.5733577958975042
# dz = dz - 5.5 (0.5179179093800803)
# dz = dz - 0.55 (0.574057730905486)
# w1: 1.33: 0.5179198059888382
# w2: 1.15: 0.5180231748280285
# test dz = -5.5 (  22 | 07m12s |    0.54280 |    1.2000 (w1) |    1.3073 (w11) |    0.7000 (w2) |    0.4000 (w3) | )
# 5 | 06m34s |    0.52901 |    1.1786 |    4.9917 |    0.6781 |    0.3283 | 

Initialization
-----------------------------------------------------------------------------
 Step |   Time |      Value |        w1 |       w11 |        w2 |        w3 | 
0.35973113260363676 0.87535329634572 0.501519736901007 0.06790236669910779 247
 179.0000000[ 0] score : 0.29131392
 179.0000000[ 1] score : 0.29652134
 179.0000000[ 2] score : 0.29913653
 179.0000000[ 3] score : 0.30038484
 179.0000000[ 4] score : 0.30055644
 179.0000000[ 5] score : 0.30136186
 179.0000000[ 6] score : 0.30131861
 179.0000000[ 7] score : 0.30144083
--------------------------------------
0.2990042953737582
    1 | 06m16s |    0.29900 |    0.3597 |    0.8754 |    0.5015 |    0.0679 | 
0.098871636440753 -4.8171879089865275 0.2560010060565079 0.23528203331116154 247
 179.0000000[ 0] score : 0.17153696
 179.0000000[ 1] score : 0.17577836
 179.0000000[ 2] score : 0.17860119
 179.0000000[ 3] score : 0.17939143
 179.0000000[ 4] score : 0.17949707
 179.0000000[ 5] score : 0.17989345
 179.0000000[ 6] score : 0.

 179.0000000[ 7] score : 0.21650514
--------------------------------------
0.21532576440603776
   16 | 06m56s |    0.21533 |    0.4694 |    4.7156 |    0.5641 |    0.3171 | 
1.1029508650994613 -2.324114387689203 0.5943814778026751 0.1995638053143997 247
 179.0000000[ 0] score : 0.33034011
 179.0000000[ 1] score : 0.33380248
 179.0000000[ 2] score : 0.33453020
 179.0000000[ 3] score : 0.33403773
 179.0000000[ 4] score : 0.33439960
 179.0000000[ 5] score : 0.33393952
 179.0000000[ 6] score : 0.33404984
 179.0000000[ 7] score : 0.33388741
--------------------------------------
0.3336233603312355
   17 | 07m19s |    0.33362 |    1.1030 |   -2.3241 |    0.5944 |    0.1996 | 
0.35260935966932616 3.4951006534177047 0.19432120884806867 0.05225098600647649 247
 179.0000000[ 0] score : 0.17897041
 179.0000000[ 1] score : 0.18066169
 179.0000000[ 2] score : 0.18279447
 179.0000000[ 3] score : 0.18411513
 179.0000000[ 4] score : 0.18447444
 179.0000000[ 5] score : 0.18457348
 179.0000000[ 6] score

 179.0000000[ 0] score : 0.48399151
 179.0000000[ 1] score : 0.48778417
 179.0000000[ 2] score : 0.49017360
 179.0000000[ 3] score : 0.49000520
 179.0000000[ 4] score : 0.49059142
 179.0000000[ 5] score : 0.49032166
 179.0000000[ 6] score : 0.49071261
 179.0000000[ 7] score : 0.49039543
--------------------------------------
0.48924694830294196
   33 | 06m36s |    0.48925 |    0.9294 |    2.7072 |    0.5663 |    0.2217 | 
0.587343096002589 1.6024700365859799 0.5086571929276598 0.16533652367402032 247
 179.0000000[ 0] score : 0.37586667
 179.0000000[ 1] score : 0.38058427
 179.0000000[ 2] score : 0.38050229
 179.0000000[ 3] score : 0.38234746
 179.0000000[ 4] score : 0.38204581
 179.0000000[ 5] score : 0.38225310
 179.0000000[ 6] score : 0.38181341
 179.0000000[ 7] score : 0.38234305
--------------------------------------
0.38096950892668013
   34 | 06m29s |    0.38097 |    0.5873 |    1.6025 |    0.5087 |    0.1653 | 
0.3480624785810137 2.5817344103278543 0.5174499319411097 0.232532081

 179.0000000[ 3] score : 0.23042650
 179.0000000[ 4] score : 0.23079245
 179.0000000[ 5] score : 0.23150962
 179.0000000[ 6] score : 0.23124617
 179.0000000[ 7] score : 0.23135988
--------------------------------------
0.22948067525383767
   49 | 06m26s |    0.22948 |    0.8624 |    2.6199 |    0.1025 |    0.1188 | 
1.0780041368839257 3.1115815039175114 0.5333928903530836 0.18705801062052285 247
 179.0000000[ 0] score : 0.50399803
 179.0000000[ 1] score : 0.50730567
 179.0000000[ 2] score : 0.50731852
 179.0000000[ 3] score : 0.50868950
 179.0000000[ 4] score : 0.50826488
 179.0000000[ 5] score : 0.50847693
 179.0000000[ 6] score : 0.50805938
 179.0000000[ 7] score : 0.50831090
--------------------------------------
0.5075529786784256
   50 | 06m10s |    0.50755 |    1.0780 |    3.1116 |    0.5334 |    0.1871 | 


/home/watts/anaconda3/envs/tmlf/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-2.83199627e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 49, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)


Bayesian Optimization
-----------------------------------------------------------------------------
 Step |   Time |      Value |        w1 |       w11 |        w2 |        w3 | 
1.2 5.5 0.03 0.4 247
 179.0000000[ 0] score : 0.21780818
 179.0000000[ 1] score : 0.22027672
 179.0000000[ 2] score : 0.22054964
 179.0000000[ 3] score : 0.22159575
 179.0000000[ 4] score : 0.22098573
 179.0000000[ 5] score : 0.22131902
 179.0000000[ 6] score : 0.22086723
 179.0000000[ 7] score : 0.22126500
--------------------------------------
0.22058340820468528
   51 | 06m35s |    0.22058 |    1.2000 |    5.5000 |    0.0300 |    0.4000 | 
1.2 -5.5 0.03 0.4 247
 179.0000000[ 0] score : 0.20612476
 179.0000000[ 1] score : 0.21485497
 179.0000000[ 2] score : 0.21648064
 179.0000000[ 3] score : 0.21704651
 179.0000000[ 4] score : 0.21666414
 179.0000000[ 5] score : 0.21734121
 179.0000000[ 6] score : 0.21764421
 179.0000000[ 7] score : 0.21721326
--------------------------------------
0.21542121259269434
   52

/home/watts/anaconda3/envs/tmlf/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([2.90769416e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 51, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)
/home/watts/anaconda3/envs/tmlf/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-1.27713042e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 54, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)


1.2 1.6400449887385833 0.7 0.4 247
 179.0000000[ 0] score : 0.49235570
 179.0000000[ 1] score : 0.49911456
 179.0000000[ 2] score : 0.50055664
 179.0000000[ 3] score : 0.50157209
 179.0000000[ 4] score : 0.50096065
 179.0000000[ 5] score : 0.50134047
 179.0000000[ 6] score : 0.50111954
 179.0000000[ 7] score : 0.50141865
--------------------------------------
0.4998047884816683
   53 | 06m29s |    0.49980 |    1.2000 |    1.6400 |    0.7000 |    0.4000 | 
0.09 5.5 0.03 0.4 247
 179.0000000[ 0] score : 0.03871106
 179.0000000[ 1] score : 0.04038705
 179.0000000[ 2] score : 0.04121413
 179.0000000[ 3] score : 0.04134795
 179.0000000[ 4] score : 0.04139673
 179.0000000[ 5] score : 0.04161959
 179.0000000[ 6] score : 0.04154660
 179.0000000[ 7] score : 0.04163473
--------------------------------------
0.04098222920333429
   54 | 05m45s |    0.04098 |    0.0900 |    5.5000 |    0.0300 |    0.4000 | 


/home/watts/anaconda3/envs/tmlf/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-1.94821387e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 48, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)


1.2 -4.416000729876377 0.7 0.4 247
 179.0000000[ 0] score : 0.38089312
 179.0000000[ 1] score : 0.39254564
 179.0000000[ 2] score : 0.39647406
 179.0000000[ 3] score : 0.39750009
 179.0000000[ 4] score : 0.39769914
 179.0000000[ 5] score : 0.39802209
 179.0000000[ 6] score : 0.39810332
 179.0000000[ 7] score : 0.39757050
--------------------------------------
0.39485099409206514
   55 | 06m47s |    0.39485 |    1.2000 |   -4.4160 |    0.7000 |    0.4000 | 


In [55]:
# def run_dbscan():

data_dir = '../data/train'

#     event_ids = [
#             '000001030',##
#             '000001025','000001026','000001027','000001028','000001029',
#     ]

event_ids = [
        '000001030',##

]

sum=0
sum_score=0
for i,event_id in enumerate(event_ids):
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    hits  = pd.read_csv(data_dir + '/event%s-hits.csv'%event_id)
    cells = pd.read_csv(data_dir + '/event%s-cells.csv'%event_id)
    truth = pd.read_csv(data_dir + '/event%s-truth.csv'%event_id)
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    
    truth = pd.merge(truth, particles, how='left', on='particle_id')
    hits = pd.merge(hits, truth, how='left', on='hit_id')
    
    w1 = [0.9, 1.2]
    w2 = [0.3, 0.7]
    w3 = [0.1, 0.4]
    Niter = [140, 190]
    
    
#     bo = BayesianOptimization(Fun4BO,pbounds = {'w1':w1,'w2':w2,'w3':w3,'Niter':Niter})
#     bo.maximize(init_points = 3, n_iter = 20, acq = "ucb", kappa = 2.576)
    w1 = 1.1932215111905984
    w2 = 0.39740553885387364
    w3 = 0.3512647720585538
    w6 = [-0.01, 0.01]
    w6 = 0.0028
    w7 = [-0.016, 0.016]
    Niter = 179
    
    bo = BayesianOptimization(Fun4BO6,pbounds = {'w7':w7})
    bo.maximize(init_points = 20, n_iter = 5, acq = "ucb", kappa = 2.576)

#     zr, dz = 0.055: 22 | 05m40s |    0.56424 |    0.0029 | 
# zr, dz = 0.0055: 12 | 05m33s |    0.56262 |    0.0033 | 
# zr, dz = 0.015: 20 | 05m36s |    0.56411 |    0.0028 | 
# zr, dz = 

Initialization
-----------------------------------------
 Step |   Time |      Value |        w7 | 
 179.0000000[ 0] score : 0.55896511
 179.0000000[ 1] score : 0.56315921
 179.0000000[ 2] score : 0.56450423
 179.0000000[ 3] score : 0.56503375
 179.0000000[ 4] score : 0.56548976
 179.0000000[ 5] score : 0.56586425
 179.0000000[ 6] score : 0.56511084
 179.0000000[ 7] score : 0.56543857
--------------------------------------
0.5641957152997237
    1 | 05m44s |    0.56420 |    0.0015 | 
 179.0000000[ 0] score : 0.55896511
 179.0000000[ 1] score : 0.56315921
 179.0000000[ 2] score : 0.56450423
 179.0000000[ 3] score : 0.56503375
 179.0000000[ 4] score : 0.56548976
 179.0000000[ 5] score : 0.56586425
 179.0000000[ 6] score : 0.56511084
 179.0000000[ 7] score : 0.56543857
--------------------------------------
0.5641957152997237
    2 | 05m39s |    0.56420 |   -0.0094 | 
 179.0000000[ 0] score : 0.55896511
 179.0000000[ 1] score : 0.56315921
 179.0000000[ 2] score : 0.56450423
 179.0000000[ 

/home/watts/anaconda3/envs/tmlf/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


Bayesian Optimization
-----------------------------------------
 Step |   Time |      Value |        w7 | 
 179.0000000[ 0] score : 0.55896511
 179.0000000[ 1] score : 0.56315921
 179.0000000[ 2] score : 0.56450423
 179.0000000[ 3] score : 0.56503375
 179.0000000[ 4] score : 0.56548976
 179.0000000[ 5] score : 0.56586425
 179.0000000[ 6] score : 0.56511084
 179.0000000[ 7] score : 0.56543857
--------------------------------------
0.5641957152997237
   21 | 05m39s |    0.56420 |    0.0145 | 


/home/watts/anaconda3/envs/tmlf/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


 179.0000000[ 0] score : 0.55896511
 179.0000000[ 1] score : 0.56315921
 179.0000000[ 2] score : 0.56450423
 179.0000000[ 3] score : 0.56503375
 179.0000000[ 4] score : 0.56548976
 179.0000000[ 5] score : 0.56586425
 179.0000000[ 6] score : 0.56511084
 179.0000000[ 7] score : 0.56543857
--------------------------------------
0.5641957152997237
   22 | 05m41s |    0.56420 |   -0.0084 | 


...

/home/watts/anaconda3/envs/tmlf/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


 179.0000000[ 0] score : 0.55896511
 179.0000000[ 1] score : 0.56315921
 179.0000000[ 2] score : 0.56450423
 179.0000000[ 3] score : 0.56503375
 179.0000000[ 4] score : 0.56548976
 179.0000000[ 5] score : 0.56586425
 179.0000000[ 6] score : 0.56511084
 179.0000000[ 7] score : 0.56543857
--------------------------------------
0.5641957152997237
   24 | 05m51s |    0.56420 |   -0.0005 | 


/home/watts/anaconda3/envs/tmlf/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


 179.0000000[ 0] score : 0.55896511
 179.0000000[ 1] score : 0.56315921
 179.0000000[ 2] score : 0.56450423
 179.0000000[ 3] score : 0.56503375
 179.0000000[ 4] score : 0.56548976
 179.0000000[ 5] score : 0.56586425
 179.0000000[ 6] score : 0.56511084
 179.0000000[ 7] score : 0.56543857
--------------------------------------
0.5641957152997237
   25 | 05m45s |    0.56420 |   -0.0044 | 


/home/watts/anaconda3/envs/tmlf/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


In [173]:
# def run_dbscan():

data_dir = '../data/train'

#     event_ids = [
#             '000001030',##
#             '000001025','000001026','000001027','000001028','000001029',
#     ]

event_ids = [
        '000001030',##

]

sum=0
sum_score=0
for i,event_id in enumerate(event_ids):
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    hits  = pd.read_csv(data_dir + '/event%s-hits.csv'%event_id)
    cells = pd.read_csv(data_dir + '/event%s-cells.csv'%event_id)
    truth = pd.read_csv(data_dir + '/event%s-truth.csv'%event_id)
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    
    truth = pd.merge(truth, particles, how='left', on='particle_id')
    hits = pd.merge(hits, truth, how='left', on='hit_id')
    
    w1 = [0.9, 1.2]
    w2 = [0.3, 0.7]
    w3 = [0.1, 0.4]
    Niter = [140, 190]
    
    
#     bo = BayesianOptimization(Fun4BO,pbounds = {'w1':w1,'w2':w2,'w3':w3,'Niter':Niter})
#     bo.maximize(init_points = 3, n_iter = 20, acq = "ucb", kappa = 2.576)
    w1 = 1.1932215111905984
    w2 = 0.39740553885387364
    w3 = 0.3512647720585538
    w4 = [0.1, 0.4]
    Niter = 179
    
    bo = BayesianOptimization(Fun4BO2,pbounds = {'w4':w4})
    bo.maximize(init_points = 5, n_iter = 10, acq = "ucb", kappa = 2.576)

#     Fun4BO2(w1,w2,w3,w4,Niter)
# 0.0000075 - 0.5489
# 0.000008 - 0.5482
# 0.0000085 - 0.5473
# 0.000009 - 0.5467
# 0.0000095 - 0.5482
# 0.00001 - 0.5467
# 0.000015 - 0.54608
# 0.00002 - 0.5495
# 0.000025 - 0.5487
# 0.000030 - 0.54811
# 0.000035 - 0.5453
# 0.00004 - 0.544969
# 0.000045 - 0.5463

Initialization
-----------------------------------------
 Step |   Time |      Value |        w4 | 
 179.0000000[ 0] score : 0.53635095
 179.0000000[ 1] score : 0.54500194
 179.0000000[ 2] score : 0.54704574
 179.0000000[ 3] score : 0.54720808
 179.0000000[ 4] score : 0.54895119
 179.0000000[ 5] score : 0.54901982
 179.0000000[ 6] score : 0.54842183
 179.0000000[ 7] score : 0.54854163
--------------------------------------
0.5463176486148132
    1 | 05m28s |    0.54632 |    0.3104 | 
 179.0000000[ 0] score : 0.54287635
 179.0000000[ 1] score : 0.54980706
 179.0000000[ 2] score : 0.55152658
 179.0000000[ 3] score : 0.55220886
 179.0000000[ 4] score : 0.55270776
 179.0000000[ 5] score : 0.55278150
 179.0000000[ 6] score : 0.55221363
 179.0000000[ 7] score : 0.55275856
--------------------------------------
0.5508600375841637
    2 | 05m24s |    0.55086 |    0.2308 | 
 179.0000000[ 0] score : 0.54685748
 179.0000000[ 1] score : 0.55368175
 179.0000000[ 2] score : 0.55467025
 179.0000000[ 

/home/watts/anaconda3/envs/tmlf/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([0.00011786]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 49, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)
/home/watts/anaconda3/envs/tmlf/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


Bayesian Optimization
-----------------------------------------
 Step |   Time |      Value |        w4 | 
 179.0000000[ 0] score : 0.54761377
 179.0000000[ 1] score : 0.55224905
 179.0000000[ 2] score : 0.55399383
 179.0000000[ 3] score : 0.55494576
 179.0000000[ 4] score : 0.55400293
 179.0000000[ 5] score : 0.55489261
 179.0000000[ 6] score : 0.55423685
 179.0000000[ 7] score : 0.55456924
--------------------------------------
0.5533130068896134
    6 | 05m26s |    0.55331 |    0.1000 | 


/home/watts/anaconda3/envs/tmlf/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


 179.0000000[ 0] score : 0.52543664
 179.0000000[ 1] score : 0.53435175
 179.0000000[ 2] score : 0.53759571
 179.0000000[ 3] score : 0.53821719
 179.0000000[ 4] score : 0.53975740
 179.0000000[ 5] score : 0.54020183
 179.0000000[ 6] score : 0.53917832
 179.0000000[ 7] score : 0.53966144
--------------------------------------
0.536800033138018
    7 | 05m34s |    0.53680 |    0.4000 | 


...

/home/watts/anaconda3/envs/tmlf/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00351778]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 65, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)
/home/watts/anaconda3/envs/tmlf/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([0.00015591]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 91, 'nit': 10, 'warnflag': 2}
  " state: %s" % convergence_dict)
/home/watts/anaconda3/envs/tmlf/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00253573]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 62, 'nit': 7, 'warnflag': 2}
  " state: %s" % convergence_dict)


 179.0000000[ 0] score : 0.54768431
 179.0000000[ 1] score : 0.55387312
 179.0000000[ 2] score : 0.55481452
 179.0000000[ 3] score : 0.55508775
 179.0000000[ 4] score : 0.55559411
 179.0000000[ 5] score : 0.55541995
 179.0000000[ 6] score : 0.55499122
 179.0000000[ 7] score : 0.55516357
--------------------------------------
0.5540785694166246
   14 | 05m35s |    0.55408 |    0.1723 | 


/home/watts/anaconda3/envs/tmlf/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00193173]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 65, 'nit': 9, 'warnflag': 2}
  " state: %s" % convergence_dict)
/home/watts/anaconda3/envs/tmlf/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00053969]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 81, 'nit': 7, 'warnflag': 2}
  " state: %s" % convergence_dict)
/home/watts/anaconda3/envs/tmlf/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([1.28565589e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 52, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)
/home/watts/anaconda3/envs/tmlf/lib/python3.6/site-package

 179.0000000[ 0] score : 0.54433602
 179.0000000[ 1] score : 0.55159334
 179.0000000[ 2] score : 0.55271159
 179.0000000[ 3] score : 0.55279125
 179.0000000[ 4] score : 0.55342873
 179.0000000[ 5] score : 0.55358803
 179.0000000[ 6] score : 0.55307841
 179.0000000[ 7] score : 0.55319152
--------------------------------------
0.5518398612533377
   15 | 05m39s |    0.55184 |    0.2093 | 


/home/watts/anaconda3/envs/tmlf/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00181406]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 81, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)
/home/watts/anaconda3/envs/tmlf/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([0.00024598]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 55, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)
/home/watts/anaconda3/envs/tmlf/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


In [ ]:
# def run_dbscan():

data_dir = '../data/train'

#     event_ids = [
#             '000001030',##
#             '000001025','000001026','000001027','000001028','000001029',
#     ]

event_ids = [
        '000001030',##

]

sum=0
sum_score=0
for i,event_id in enumerate(event_ids):
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    hits  = pd.read_csv(data_dir + '/event%s-hits.csv'%event_id)
    cells = pd.read_csv(data_dir + '/event%s-cells.csv'%event_id)
    truth = pd.read_csv(data_dir + '/event%s-truth.csv'%event_id)
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    
    truth = pd.merge(truth, particles, how='left', on='particle_id')
    hits = pd.merge(hits, truth, how='left', on='hit_id')
    
    w1 = [0.9, 1.2]
    w2 = [0.3, 0.7]
    w3 = [0.1, 0.4]
    Niter = [140, 190]
    
    
    bo = BayesianOptimization(Fun4BO,pbounds = {'w1':w1,'w2':w2,'w3':w3,'Niter':Niter})
    bo.maximize(init_points = 3, n_iter = 20, acq = "ucb", kappa = 2.576)
    

Initialization
-----------------------------------------------------------------------------
 Step |   Time |      Value |     Niter |        w1 |        w2 |        w3 | 
184.7827709130419
184
1.0561046005761139 0.5351622558071004 0.3762123539827661 184
 179.0000000[ 0] score : 0.53942962
 179.0000000[ 1] score : 0.54606243
 179.0000000[ 2] score : 0.54780086
 179.0000000[ 3] score : 0.54875295
 179.0000000[ 4] score : 0.54815666
 179.0000000[ 5] score : 0.54860441
 179.0000000[ 6] score : 0.54801785
 179.0000000[ 7] score : 0.54881484
--------------------------------------
0.5469549540012781
    1 | 05m08s |    0.54695 |  184.7828 |    1.0561 |    0.5352 |    0.3762 | 
145.0794227877262
145
1.1353520277193074 0.658195945930628 0.24032322705809864 145
 179.0000000[ 0] score : 0.52382250
 179.0000000[ 1] score : 0.52837456
 179.0000000[ 2] score : 0.53034994
 179.0000000[ 3] score : 0.53139514
 179.0000000[ 4] score : 0.53153080
 179.0000000[ 5] score : 0.53243534
 179.0000000[ 6] scor

 179.0000000[ 2] score : 0.45956269
 179.0000000[ 3] score : 0.46153693
 179.0000000[ 4] score : 0.46225305
 179.0000000[ 5] score : 0.46312552
 179.0000000[ 6] score : 0.46227716
 179.0000000[ 7] score : 0.46323152
--------------------------------------
0.46055974873573446
   17 | 05m21s |    0.46056 |  186.4388 |    1.2000 |    0.3000 |    0.1000 | 
175.93802446513746
175
1.2 0.7 0.4 175
 179.0000000[ 0] score : 0.52847339
 179.0000000[ 1] score : 0.53424707
 179.0000000[ 2] score : 0.53617756
 179.0000000[ 3] score : 0.53704935
 179.0000000[ 4] score : 0.53663841
 179.0000000[ 5] score : 0.53773147
 179.0000000[ 6] score : 0.53728202
 179.0000000[ 7] score : 0.53720027
--------------------------------------
0.5355999427729057
   18 | 05m21s |    0.53560 |  175.9380 |    1.2000 |    0.7000 |    0.4000 | 
164.75000708655898
164
0.9 0.7 0.4 164
 179.0000000[ 0] score : 0.52636347
 179.0000000[ 1] score : 0.53308124
 179.0000000[ 2] score : 0.53533614
 179.0000000[ 3] score : 0.53587774

In [64]:
# def run_dbscan():

data_dir = '../data/train'

#     event_ids = [
#             '000001030',##
#             '000001025','000001026','000001027','000001028','000001029',
#     ]

event_ids = [
        '000001030',##

]

sum=0
sum_score=0
for i,event_id in enumerate(event_ids):
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    hits  = pd.read_csv(data_dir + '/event%s-hits.csv'%event_id)
    cells = pd.read_csv(data_dir + '/event%s-cells.csv'%event_id)
    truth = pd.read_csv(data_dir + '/event%s-truth.csv'%event_id)
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    
    truth = pd.merge(truth, particles, how='left', on='particle_id')
    hits = pd.merge(hits, truth, how='left', on='hit_id')
    
    w1 = [0.9, 1.2]
    w2 = [0.3, 0.7]
    w3 = [0.1, 0.4]
    Niter = [140, 190]
    
    
#     bo = BayesianOptimization(Fun4BO,pbounds = {'w1':w1,'w2':w2,'w3':w3,'Niter':Niter})
#     bo.maximize(init_points = 3, n_iter = 20, acq = "ucb", kappa = 2.576)
    w1 = 1.1932215111905984
    w2 = 0.39740553885387364
    w3 = 0.3512647720585538
    Niter = 179
    
    Fun4BO(w1,w2,w3,Niter)

 179.0000000[ 0] score : 0.54152670
 179.0000000[ 1] score : 0.54799279
 179.0000000[ 2] score : 0.54956076
 179.0000000[ 3] score : 0.55086577
 179.0000000[ 4] score : 0.54973054
 179.0000000[ 5] score : 0.55075552
 179.0000000[ 6] score : 0.54987083
 179.0000000[ 7] score : 0.55092164
--------------------------------------
0.548903067437077


In [175]:
w1 = 1.1932215111905984
w2 = 0.39740553885387364
w3 = 0.3512647720585538
w4 = 0.1574 # 0.55590
Niter = 179

# 172.3593 |    1.0579 |    0.5147 |    0.3853
# 1.1932215111905984 0.39740553885387364 0.3512647720585538 179

In [24]:
w1 = 1.1932215111905984
w2 = 0.39740553885387364
w3 = 0.3512647720585538
w4 = 0.1470
w5 = 0.01201
w6 = 0.0003864
w7 = 0.0205
w8 = 0.0049
w9 = 0.00121
w10 = 1.4930496676654575e-05
Niter = 247

In [25]:
def run_make_submission():

    data_dir = '../data/test'


    tic = t = time.time()
    event_ids = [ '%09d'%i for i in range(0,125) ]  #(0,125)

    if 1:
        submissions = []
        for i,event_id in enumerate(event_ids):
            hits  = pd.read_csv(data_dir + '/event%s-hits.csv'%event_id)
            cells = pd.read_csv(data_dir + '/event%s-cells.csv'%event_id)

#             track_id = trackML4(hits, w1, w2, w3, w4, Niter)
            track_id = trackML31(hits, w1, w2, w3, w4, w5, w6, w7, w8, w9, w10, Niter)
            #track_id = back_fit(track_id,hits)

            toc =  time.time()
            print('\revent_id : %s  , %0.0f min'%(event_id, (toc-tic)/60))

            # Prepare submission for an event
            submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
                data=np.column_stack(([event_id,]*len(hits), hits.hit_id.values, track_id))
            ).astype(int)
            submissions.append(submission)
            
            for i in range(8):
                submission = extend(submission,hits)
            
            
            submission.to_csv('../cache/sub2/%s.csv.gz'%event_id,
                                index=False, compression='gzip')

            #------------------------------------------------------
    if 1:

        event_ids = [ '%09d'%i for i in range(0,125) ]  #(0,125)
        submissions = []
        for i,event_id in enumerate(event_ids):
            submission  = pd.read_csv('../cache/sub2/%s.csv.gz'%event_id, compression='gzip')
            submissions.append(submission)

        
        # Create submission file
        submission = pd.concat(submissions, axis=0)
        submission.to_csv('../submissions/sub2/submission-0024.csv.gz',
                            index=False, compression='gzip')
        print(len(submission))

In [26]:
run_make_submission()

event_id : 000000000  , 6 min
event_id : 000000001  , 15 min
event_id : 000000002  , 23 min
event_id : 000000003  , 31 min
event_id : 000000004  , 39 min
event_id : 000000005  , 47 min
event_id : 000000006  , 55 min
event_id : 000000007  , 63 min
event_id : 000000008  , 72 min
event_id : 000000009  , 80 min
event_id : 000000010  , 88 min
event_id : 000000011  , 97 min
event_id : 000000012  , 104 min
event_id : 000000013  , 114 min
event_id : 000000014  , 122 min
event_id : 000000015  , 129 min
event_id : 000000016  , 136 min
event_id : 000000017  , 146 min
event_id : 000000018  , 154 min
event_id : 000000019  , 162 min
event_id : 000000020  , 169 min
event_id : 000000021  , 176 min
event_id : 000000022  , 185 min
event_id : 000000023  , 194 min
event_id : 000000024  , 203 min
event_id : 000000025  , 211 min
event_id : 000000026  , 219 min
event_id : 000000027  , 226 min
event_id : 000000028  , 234 min
event_id : 000000029  , 241 min
event_id : 000000030  , 249 min
event_id : 000000031 